In [1]:
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:

    sys.path.append(project_root)
    sys.path.append( os.path.join(project_root , 'src'))

%load_ext autoreload
%autoreload 2


from gee_utils import (
    walk_assets,
    make_tree_public,
    share_tree_with_user,
)


In [2]:
import ee, geetools
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [3]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.gee_utils import (
    upload_to_gee,
    check_asset_types,
    inspect_asset_metadata,
    count_assets_in_folder,
    create_gee_folder,
    delete_all_subfolders,
    make_assets_public,
    walk_assets,
    make_tree_public,
)

In [4]:
import os

# Local folder with GeoTIFFs
local_folder = "../data/OkavangoDelta_flooding-master/Rasters/Annual_inundation_maps"

# GEE asset folder
gee_asset_folder = "projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps"

# List all .tif files
tif_files = [f for f in os.listdir(local_folder) if f.endswith(".tif")]

for tif in tif_files[:5]:
    local_path = os.path.join(local_folder, tif)
    upload_to_gee(local_path, gee_asset_folder, execute=False)


Source: ../data/OkavangoDelta_flooding-master/Rasters/Annual_inundation_maps/InundationMap_2008.tif
Asset:  projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps/InundationMap_2008
Source: ../data/OkavangoDelta_flooding-master/Rasters/Annual_inundation_maps/InundationMap_2018.tif
Asset:  projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps/InundationMap_2018
Source: ../data/OkavangoDelta_flooding-master/Rasters/Annual_inundation_maps/InundationMap_2019.tif
Asset:  projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps/InundationMap_2019
Source: ../data/OkavangoDelta_flooding-master/Rasters/Annual_inundation_maps/InundationMap_1991.tif
Asset:  projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps/InundationMap_1991
Source: ../data/OkavangoDelta_flooding-master/Rasters/Annual_inundation_maps/InundationMap_1990.tif
Asset:  projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps/InundationMap_1990


In [8]:
asset_list = [d['id'] for d in 
              ee.data.listAssets("projects/ee-okavango/assets/water_masks")['assets']]

asset_list

['projects/ee-okavango/assets/water_masks/Lungwebungu_Sent2_DSWE_monthly_10m',
 'projects/ee-okavango/assets/water_masks/Moxico_Sent2_DSWE_monthly_10m',
 'projects/ee-okavango/assets/water_masks/Sudd_Sent2_DSWE_monthly_10m',
 'projects/ee-okavango/assets/water_masks/composites',
 'projects/ee-okavango/assets/water_masks/lukas_binarized_months',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2',
 'projects/ee-okavango/assets/water_masks/sampled_DSWE',
 'projects/ee-okavango/assets/water_masks/sentinel_2_dswe_5day_composites',
 'projects/ee-okavango/assets/water_masks/yearly_DSWE_Sent2_10m_winnie']

In [9]:
asset_path = 'projects/ee-okavango/assets/shapes/testing_grounds_20241127'
asset = ee.FeatureCollection(asset_path)

# Get information about the asset
info = asset.getInfo()

info

{'type': 'FeatureCollection',
 'columns': {'system:index': 'String'},
 'version': 1733849180036364,
 'id': 'projects/ee-okavango/assets/shapes/testing_grounds_20241127',
 'properties': {'system:asset_size': 7369},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[22.969779905690068, -19.103550567786034],
      [22.982804225398247, -19.33262114707843],
      [23.180895834770727, -19.336720261980165],
      [23.17084528571743, -19.2183413491786],
      [23.160818226000984, -19.099959096000124],
      [22.969779905690068, -19.103550567786034]]]},
   'id': '00000000000000000000',
   'properties': {}}]}

In [15]:

def set_folder_permissions(folder_path, email, role="READER"):
    # List all assets in the folder
    asset_list = ee.data.listAssets({'parent': folder_path})['assets']
    

    for asset in asset_list:
        asset_id = asset['id']
        # Get current access control list (ACL) for the asset
        acl = ee.data.getAssetAcl(asset_id)
        
        # Update ACL to share with the new user
        acl['owners'] = acl.get('owners', [])
        acl['writers'] = acl.get('writers', [])
        acl['readers'] = acl.get('readers', [])
        
        if role == "READER":
            acl['readers'].append(f"user:{email}")  # Add the user prefix
        elif role == "WRITER":
            
            acl['writers'].append(f"user:{email}")  # Add the user prefix
        print (acl )
        
        # Set the new ACL for the asset
        ee.data.setAssetAcl(asset_id, acl)

# Example usage
folder_path = "projects/ee-quatratavia/assets/berms/DEP3"
user_email = 'quatratavia@gmail.com'  # Email of the user you want to share with
set_folder_permissions(folder_path, user_email, role="WRITER")  # Role can be "READER" or "WRITER"


{'owners': [], 'readers': ['user:aaronkoop@gmail.com', 'user:quatratavia@gmail.com'], 'writers': ['user:quatratavia@gmail.com', 'user:quatratavia@gmail.com'], 'all_users_can_read': True}
{'owners': [], 'readers': ['user:aaronkoop@gmail.com', 'user:quatratavia@gmail.com'], 'writers': ['user:quatratavia@gmail.com', 'user:quatratavia@gmail.com']}
{'owners': [], 'readers': ['user:aaronkoop@gmail.com', 'user:quatratavia@gmail.com'], 'writers': ['user:quatratavia@gmail.com', 'user:quatratavia@gmail.com']}
{'owners': [], 'readers': ['user:aaronkoop@gmail.com', 'user:quatratavia@gmail.com'], 'writers': ['user:quatratavia@gmail.com', 'user:quatratavia@gmail.com']}
{'owners': [], 'readers': ['user:aaronkoop@gmail.com', 'user:quatratavia@gmail.com'], 'writers': ['user:quatratavia@gmail.com', 'user:quatratavia@gmail.com']}
{'owners': [], 'readers': ['user:aaronkoop@gmail.com', 'user:quatratavia@gmail.com'], 'writers': ['user:quatratavia@gmail.com', 'user:quatratavia@gmail.com']}
{'owners': [], 're

{'owners': [], 'readers': ['user:aaronkoop@gmail.com', 'user:quatratavia@gmail.com'], 'writers': ['user:quatratavia@gmail.com']}
{'owners': [], 'readers': ['user:aaronkoop@gmail.com', 'user:quatratavia@gmail.com'], 'writers': ['user:quatratavia@gmail.com']}
{'owners': [], 'readers': ['user:aaronkoop@gmail.com', 'user:quatratavia@gmail.com'], 'writers': ['user:quatratavia@gmail.com']}
{'owners': [], 'readers': ['user:aaronkoop@gmail.com', 'user:quatratavia@gmail.com'], 'writers': ['user:quatratavia@gmail.com']}


In [16]:
# Code to move assets around
src = ee.Asset("projects/ee-quatratavia/assets/monthly_LST")
dst = ee.Asset("projects/ee-quatratavia/assets/CA_fire/monthly_LST")

# src.move(dst)


In [17]:
src = ee.Asset("projects/ee-quatratavia/assets/MBTS")
dst = ee.Asset("projects/ee-quatratavia/assets/CA_fire/MBTS")

# src.move(dst)


In [18]:
# Polygon-based conditional deletion with safety & dry-run
import ee

def delete_images_outside_polygon(folder_path: str, polygon_fc_path: str, dry_run: bool = True, limit: int | None = None):
    """Delete (optionally dry-run) images in a folder whose geometry lies completely outside a polygon.

    Args:
        folder_path: GEE folder containing image assets.
        polygon_fc_path: FeatureCollection asset containing at least one feature (first used as polygon).
        dry_run: If True, only print planned deletions.
        limit: If set, only evaluate up to this many assets (debug / safety).
    """
    ee.Initialize()
    try:
        polygon_fc = ee.FeatureCollection(polygon_fc_path).first()
        if polygon_fc is None:
            print("Polygon FeatureCollection is empty; aborting.")
            return
        polygon_geometry = polygon_fc.geometry()
    except Exception as e:
        print(f"Failed to load polygon: {e}")
        return

    try:
        assets = ee.data.listAssets({'parent': folder_path}).get('assets', [])
    except Exception as e:
        print(f"Failed to list assets in {folder_path}: {e}")
        return

    if limit is not None:
        assets = assets[:limit]

    to_delete = []
    kept = 0
    for asset in assets:
        aid = asset.get('name') or asset.get('id')
        if not aid:
            continue
        try:
            img = ee.Image(aid)
            img_geom = img.geometry()
            outside = ee.Algorithms.IsEqual(img_geom.intersection(polygon_geometry, 1).area(), 0).getInfo()
            if outside:
                to_delete.append(aid)
            else:
                kept += 1
        except Exception as e:
            print(f"Skipping {aid}: {e}")

    print(f"Planned deletions: {len(to_delete)} | Kept (inside): {kept}")
    for aid in to_delete:
        print(f"DELETE -> {aid}")

    if dry_run:
        print("Dry-run mode, no deletions executed.")
        return

    for aid in to_delete:
        try:
            ee.data.deleteAsset(aid)
            print(f"Deleted {aid}")
        except Exception as e:
            print(f"Failed to delete {aid}: {e}")

# Example usage (dry-run)
# delete_images_outside_polygon(
#     folder_path="projects/ee-okavango/assets/dsms",
#     polygon_fc_path="projects/ee-okavango/assets/shapes/testing_grounds_20241127",
#     dry_run=True,
#     limit=25,
# )


In [19]:
# Share all assets under a root with a specific user (dry-run first)
from src.gee_utils import share_tree_with_user

ROOT = "projects/ee-okavango/assets/dsms"
EMAIL = "user@example.com"  # or use 'group:my-group@domain' for groups

# Dry run: prints what would change
share_tree_with_user(ROOT, EMAIL, role='READER', dry_run=True)

# When ready to apply, set dry_run=False
# share_tree_with_user(ROOT, EMAIL, role='READER', dry_run=False, sleep_sec=0.1)


Failed to list assets under projects/ee-okavango/assets/dsms: Asset "projects/ee-okavango/assets/dsms" not found.
FAILED: projects/ee-okavango/assets/dsms -> Asset 'gcp_projects/956467029386/dsms' does not exist or doesn't allow this operation.
